In [ ]:
import gzip
import numpy as np
import time
import pandas as pd
import h5py
import json
import os
import pandas as pd
import time
from matplotlib import pyplot as plt
import seaborn as sns
sns.set()

def Gauss3(x, a0, x0, s0):
    import numpy as np
    return a0 * np.exp(-(x - x0)**2 / (2 * s0**2))
def Gauss4(x, a0, x0, s0, d0):
    import numpy as np
    return a0 * np.exp(-(x - x0)**2 / (2 * s0**2))+d0
def Line(x, m, q):
    import numpy as np
    return m*x + q


def PMT_fit(t, tau_s, tau, t0, a0, d0):
    import numpy as np
    #
    # a0 = G*N*e*R G ~ 10^6 guadagno, N~ 100 dinodi, R resistenza e carica elettrone
    # William_R_Leo_Techniques_for_nuclear_and_partic.pdf pag 182 formula 8.13
    # 
    if abs(tau-tau_s) < 1e-5:
        V = np.heaviside(t-t0, 0)*(a0/(tau_s**2))*(t-t0)*(np.exp(-(t-t0)/tau_s))+d0
    else:
        V = -np.heaviside(t-t0, 0)*(a0/(tau-tau_s))*(np.exp(-(t-t0)/tau_s)-np.exp(-(t-t0)/tau))+d0
    return V

def fittalo(func, x, y, p0, ax, fmt='k'):
    from scipy.optimize import curve_fit
    from sklearn.metrics import r2_score
    popt, pcov = curve_fit(func,x, y, p0=p0)
    perr = np.sqrt(np.diag(pcov))
    r2=r2_score(y, func(x, *popt))
    xf = np.linspace(x.min(), x.max(), 100)
    if not ax=='':
        ax.plot(xf, func(xf, *popt), fmt, label='a = {0:.2e}±{1:.2e}\nb = {2:.2e}±{3:.2e} \
        \nc = {4:.2e}±{5:.2e}\nr^2 = {6:.3f}'.format(popt[0], perr[0], popt[1], perr[1], popt[2], perr[2], r2))
    return ax, popt, perr


def time_fit(x, y, val, ax='', col=''):
    import numpy as np

    if val ==-1:
        ax, par, err = fittalo(PMT_fit, x, y, [10,100, x[y.argmin()]-10, y.min()*20, y[0:80].mean()], ax=ax, fmt='k--')
    elif val==1:
        ax, par, err = fittalo(PMT_fit, x, y, [5,10, x[y.argmin()]-10, y.min()*20, y[0:80].mean()], ax=ax, fmt='k--')
    xmin = int(par[2])
    xmax = int(par[2]+7*par[1]*np.log(2))
    if xmax>x[-1]:
        xmax=x[-1]
    if not ax=='':
        ax.plot(x, y, col)
#         ax.axvline(xmin)
#         ax.axvline(xmax)
    # tau_s, tau, t0, a0, d0
    return par, err, np.trapz(y[xmin:xmax], x[xmin:xmax]), ax

def decompress_run(run, path='/jupyter-workspace/cloud-storage/cygno-analysis/WC/'):
    import os
    import platform
    hostos = platform.system()
    filename = 'run_{:05d}.h5'.format(run)
    filepath = path+filename
    if os.path.isfile(filepath):
        print('file ready')
    else:
        if os.path.isfile(filepath+'.gz'):
            print('decompressing {:}...'.format(filename))
            if hostos=='Darwin':
                os.system('gunzip --keep ' + filepath)
            elif hostos=='Linux': 
                os.system('gunzip -c '+filepath+'.gz > '+filepath)
            else:
                print('note recognized OS')
        else:
            print('no compressed file')
        print('decompession done')
    print('all done, ready')
    return filename

def remove_when_done(run, path='/jupyter-workspace/cloud-storage/cygno-analysis/WC/'):
    import os
    filename = 'run_{:05d}.h5'.format(run)
    filepath = path+filename
    if not os.path.isfile(filepath+'.gz'):
        print("compressing...") 
        os.system('gzip ' + filepath)
        if os.path.isfile(filepath+'.gz'):
            print("deleting uncompressed") 
            os.remove(filepath)
            print("file uncompressed removed, after compression")
        else:
            print("ERROR")
    else:
        if os.path.isfile(filepath):
            print("deleting uncompressed") 
            os.remove(filepath)
            print("file uncompressed removed")
        else:
            print("ERROR")
    return filename

def time_fft(x,y, col, label):
#    fig, ax = plt.subplots(1,2, figsize=(10,4))
    ax[0].plot(x,y, col, label=label)
    ax[0].grid()
    ax[0].legend()

    signalFFT = np.fft.rfft(y)
    signalPSD = np.abs(signalFFT) ** 2
    fftFreq = np.fft.fftfreq(len(signalPSD))
    i = fftFreq>0
    A = 10*np.log10(signalPSD[i])
    ax[1].plot(fftFreq[i], A, col, label=label)
    ax[1].grid()
    ax[1].set_xlim(0,0.1)
    ax[1].legend()
    #plt.show()